In [1]:
from datetime import timedelta as delta
from os import path
from glob import glob
import numpy as np
import dask
import math
import xarray as xr
from netCDF4 import Dataset
import warnings
import matplotlib.pyplot as plt
import pickle
warnings.simplefilter('ignore', category=xr.SerializationWarning)
from operator import attrgetter

from parcels import AdvectionRK4
from parcels import Field
from parcels import FieldSet
from parcels import JITParticle, ScipyParticle
from parcels import ParticleFile
from parcels import ParticleSet
from parcels import Variable
from parcels import VectorField
from parcels import ErrorCode

In [2]:
#input
wstokes = False          #False || True
#data_in_waves = "/projects/0/topios/hydrodynamic_data"
data_in_mit = "../../input/modelfields/MITgcm4km"
data_out = "../../input/particles"
filename_out = "Unbeaching_200810"
galapagos_domain = [-94, -87, -3.5, 3]
seeding_distance = 1 #unit: lon/lat degree
seeding_resolution = 4 #unit: gridpoints
seeding_frequency = 5 #unit: days
advection_duration = 90 #unit: days
output_frequency = 6 #unit: hours
length_simulation = 4*365 #unit: days

#Get indices for Galapagos domain to run simulation
def getclosest_ij(lats,lons,latpt,lonpt):    
    """Function to find the index of the closest point to a certain lon/lat value."""
    dist_lat = (lats-latpt)**2                      # find squared distance of every point on grid
    dist_lon = (lons-lonpt)**2
    minindex_lat = dist_lat.argmin()                # 1D index of minimum dist_sq element
    minindex_lon = dist_lon.argmin()
    return minindex_lat, minindex_lon                # Get 2D index for latvals and lonvals arrays from 1D index

dfile = Dataset(data_in_mit+'/RGEMS3_Surf_grid.nc')
lon = dfile.variables['XG'][:]
lat = dfile.variables['YG'][:]
iy_min, ix_min = getclosest_ij(lat, lon, galapagos_domain[2], galapagos_domain[0])
iy_max, ix_max = getclosest_ij(lat, lon, galapagos_domain[3], galapagos_domain[1])

#Load distance and seaborder map
file = open('distance_map', 'rb')
data_distance = pickle.load(file)
file.close()
file = open('seaborder_map', 'rb')
data_seaborder = pickle.load(file)
file.close()
lat_high = data_distance['lat']
lon_high = data_distance['lon']
distance_map = data_distance['distance']
seaborder_map = data_seaborder['seaborder']

In [3]:
# add MITgcm field
varfiles = sorted(glob(data_in_mit + "/RGEMS_20*.nc"))
meshfile = glob(data_in_mit+"/RGEMS3_Surf_grid.nc")
files_MITgcm = {'U': {'lon': meshfile, 'lat': meshfile, 'data': varfiles},
                'V': {'lon': meshfile, 'lat': meshfile, 'data': varfiles}}
variables_MITgcm = {'U': 'UVEL', 'V': 'VVEL'}
dimensions_MITgcm = {'lon': 'XG', 'lat': 'YG', 'time': 'time'}
indices_MITgcm = {'lon': range(ix_min,ix_max), 'lat': range(iy_min,iy_max)}

fieldset_MITgcm = FieldSet.from_mitgcm(files_MITgcm,
                                       variables_MITgcm, 
                                       dimensions_MITgcm, 
                                       indices = indices_MITgcm,)
fieldset = fieldset_MITgcm

/Users/f103457/parcels/parcels/field.py:338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  timeslices = np.array(timeslices)
/Users/f103457/parcels/parcels/field.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dataFiles = np.concatenate(np.array(dataFiles))


In [4]:
# add unbeaching field

file_UnBeach = 'UnbeachingUV.nc'
variables_UnBeach = {'U_unbeach': 'unBeachU',
                     'V_unbeach': 'unBeachV'}
dimensions_UnBeach = {'lon': 'XG', 'lat': 'YG'}
fieldset_UnBeach = FieldSet.from_c_grid_dataset(file_UnBeach, 
                                                variables_UnBeach,
                                                dimensions_UnBeach,
                                                indices = indices_MITgcm,
                                                tracer_interp_method='cgrid_velocity')
fieldset.add_field(fieldset_UnBeach.U_unbeach)
fieldset.add_field(fieldset_UnBeach.V_unbeach)

UVunbeach = VectorField('UVunbeach', fieldset.U_unbeach, fieldset.V_unbeach)
fieldset.add_vector_field(UVunbeach)

In [5]:
# add distance and seaborder map

fieldset.add_field(Field('distance', 
                         data = distance_map,
                         lon = lon_high,
                         lat = lat_high,                   
                         mesh='spherical',
                         interp_method = 'nearest'))
                   
fieldset.add_field(Field('island', 
                         data = seaborder_map,
                         lon = lon_high,
                         lat = lat_high,                   
                         mesh='spherical',
                         interp_method = 'nearest'))

In [6]:
# get all lon, lat that are land
fU=fieldset_MITgcm.U
fieldset_MITgcm.computeTimeChunk(fU.grid.time[0], 1)
lon = np.array(fU.lon[:]) 
lat = np.array(fU.lat[:])
LandMask = fU.data[0,:,:]
LandMask = np.array(LandMask)
land = np.where(LandMask == 0)

# seed particles at seeding_distance from land
lons = np.array(fU.lon[::seeding_resolution])
lats = np.array(fU.lat[::seeding_resolution])
yy, xx = np.meshgrid(lats,lons)
xcoord = np.reshape(xx,len(lons)*len(lats))
ycoord = np.reshape(yy,len(lons)*len(lats))

startlon=[]
startlat=[]

for i in range(xcoord.shape[0]):
    dist = (xcoord[i]-lon[land[1]])**2 + (ycoord[i]-lat[land[0]])**2
    minindex = dist.argmin()
    if dist[minindex]<seeding_distance and dist[minindex] != 0:
        startlon.append(xcoord[i])
        startlat.append(ycoord[i])

In [7]:
#functions to add to the kernel

def AdvectionRK4(particle, fieldset, time):
    if particle.beached == 0:
        (u1, v1) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)

        (u2, v2) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat1, lon1]
        lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)

        (u3, v3) = fieldset.UV[time + .5 * particle.dt, particle.depth, lat2, lon2]
        lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)

        (u4, v4) = fieldset.UV[time + particle.dt, particle.depth, lat3, lon3]
        particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
        particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
        particle.beached = 2
        
def BeachTesting(particle, fieldset, time):
    if particle.beached == 2:
        (u, v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        particle.uvel = u
        particle.vvel = v
        if fabs(u) < 1e-14 and fabs(v) < 1e-14:
            particle.beached = 1
        else:
            particle.beached = 0

def UnBeaching(particle, fieldset, time):
    if particle.beached == 1:
        (ub, vb) = fieldset.UVunbeach[time, particle.depth, particle.lat, particle.lon]
        particle.lon += ub * particle.dt * 400/particle.dt
        particle.lat += vb * particle.dt * 400/particle.dt
        particle.beached = 0
        particle.unbeachCount += 1

def UnBeaching2(particle, fieldset, time):
    if particle.beached == 1:
        particle.lon = particle.prevlon
        particle.lat = particle.prevlat
        particle.beached = 0
        particle.unbeachCount += 1
    particle.prevlon = particle.lon
    particle.prevlat = particle.lat
        
def Age(fieldset, particle, time):
    particle.age = particle.age + math.fabs(particle.dt)
    if particle.age > 90*86400:
        particle.delete()
    
def SampleInfo(fieldset, particle, time):
    particle.distance = fieldset.distance[time, particle.depth, particle.lat, particle.lon]
    particle.island = fieldset.island[time, particle.depth, particle.lat, particle.lon]

def DeleteParticle(particle, fieldset, time):
    particle.delete()
      
class GalapagosParticle(JITParticle):
    age = Variable('age', dtype=np.float32, initial = 0.)
    beached = Variable('beached', dtype=np.int32, to_write=False, initial = 0.)
    unbeachCount = Variable('unbeachCount', dtype=np.int32, initial = 0.)
    distance = Variable('distance', dtype=np.float32, initial = 0.)
    island = Variable('island', dtype=np.int32, initial = 0.)
    prevlon = Variable('prevlon', dtype=np.float32, to_write=False, initial = attrgetter('lon'))
    prevlat = Variable('prevlat', dtype=np.float32, to_write=False, initial = attrgetter('lat'))
    vvel = Variable('vvel', dtype=np.float32, initial = 0.)
    uvel = Variable('uvel', dtype=np.float32, initial = 0.)

In [8]:
# set particle conditions
pset = ParticleSet(fieldset=fieldset,
                   pclass=GalapagosParticle,
                   lon=startlon,
                   lat=startlat,
                   repeatdt=delta(days=seeding_frequency))

beaching_kernel = 'usingUV'
kernel = pset.Kernel(AdvectionRK4) + pset.Kernel(BeachTesting)
if beaching_kernel == 'usingUV':
    kernel += pset.Kernel(UnBeaching) 
    fname = path.join(data_out, filename_out + "_Test.nc") 
else:
    kernel += pset.Kernel(UnBeaching2)
    fname = path.join(data_out, filename_out + "_PrevPosition.nc") 
kernel += pset.Kernel(SampleInfo) + pset.Kernel(Age)
  
outfile = pset.ParticleFile(name=fname, outputdt=delta(hours=output_frequency))

#pset.execute(kernel,
#             runtime=delta(days=length_simulation),
#             dt=delta(hours=1),
#             output_file=outfile,
#             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

pset.repeatdt = None

pset.execute(kernel,
             runtime=delta(days=advection_duration),
             dt=delta(hours=1),
             output_file=outfile,
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

outfile.export()
outfile.close()  

INFO: Compiled GalapagosParticleAdvectionRK4BeachTestingUnBeachingSampleInfoAge ==> /var/folders/8h/ddqcss_n7hdgcs50lm1s7z_c0000gn/T/parcels-501/d4df17a4239e5635ab704ae2386b8890_0.so
INFO: Temporary output files are stored in ../../input/particles/out-PJCXJPTJ.
INFO: You can use "parcels_convert_npydir_to_netcdf ../../input/particles/out-PJCXJPTJ" to convert these to a NetCDF file during the run.
100% (7776000.0 of 7776000.0) |##########| Elapsed Time: 0:00:04 Time:  0:00:04
